In [1]:
import pandas as pd
test_df=pd.read_parquet('../test.parquet')
test_df

,id,dates,values
0,6125,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[1.85, -0.04, 0.19, -0.45, -0.75, -0.95, -2.91..."
1,26781,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[-0.41, 0.39, -0.47, -0.9, -1.46, -0.51, 0.51,..."
2,13333,"[2016-06-01, 2016-07-01, 2016-08-01, 2016-09-0...","[-0.29, -1.26, 0.17, -1.22, 0.45, -0.94, 0.16,..."
3,53218,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[-1.47, 1.55, -0.03, 0.57, -0.57, 0.6, 0.27, 1..."
4,84204,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[2.33, 1.39, -1.03, -2.64, 1.89, 1.77, 1.43, 1..."
...,...,...,...
19995,80341,"[2016-03-01, 2016-04-01, 2016-05-01, 2016-06-0...","[3.01, -0.58, 1.55, 0.48, -0.35, 1.93, 3.86, 2..."
19996,5891,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[0.66, 1.3, 2.78, -0.25, -1.97, -0.55, -1.08, ..."
19997,29091,"[2017-01-01, 2017-02-01, 2017-03-01, 2017-04-0...","[0.09, 0.44, 1.55, 0.15, 0.3, 0.19, 0.34, 1.05..."
19998,85877,"[2017-04-01, 2017-05-01, 2017-06-01, 2017-07-0...","[0.28, 1.92, 1.14, 2.4, 1.46, 1.08, -0.12, 0.6..."


In [5]:
def explode_columns(df):
    a=['dates','values']
    return df.explode(a).reset_index(drop=True)

In [7]:
test=test_df.copy()
batch_size=10000
n_batches=len(test)//batch_size
all_batch=[]
for i in range(n_batches):
    batch=test.iloc[i*batch_size:+i*batch_size+batch_size]
    batch_explode=explode_columns(batch)
    batch_explode['dates']=pd.to_datetime(batch_explode['dates'])
    batch_explode['year'] = batch_explode['dates'].dt.year
    avg_batch=batch_explode.groupby(['id', 'year'])['values'].mean().reset_index()
    all_batch.append(avg_batch)
test=pd.concat(all_batch)
test['values']=test['values'].astype(float)

In [8]:
test

,id,year,values
0,6,2016,-0.069167
1,6,2017,-0.386667
2,6,2018,0.135000
3,6,2019,0.643333
4,6,2020,-0.927853
...,...,...,...
59540,99980,2016,-0.095000
59541,99980,2017,-0.452500
59542,99980,2018,0.055000
59543,99980,2019,-0.065000


In [11]:
import joblib
loaded_model = joblib.load('model.pkl')

In [15]:
predictions = loaded_model.predict_proba(test)[:,0]

In [29]:
submission = pd.DataFrame({
    'id': test['id'],        
    'score': predictions
})

In [34]:
submission=submission.groupby('id').agg({'score':'mean'})

In [36]:
submission=submission.reset_index()
submission

,id,score
0,2,0.669364
1,6,0.742865
2,7,0.727407
3,13,0.799855
4,17,0.735789
...,...,...
19995,99965,0.535721
19996,99975,0.770829
19997,99980,0.754828
19998,99981,0.679401


In [38]:
submission.to_csv('submission.csv', index=False)